In [1]:
import numpy as np
import scipy.sparse as sp
import pandas as pd
import numba as nb

In [2]:
argl = [i.rstrip("\n") for i in open("proj1_parameter-file.txt", "r")]
argl

['MIS(10) = 0.43',
 'MIS(20) = 0.30',
 'MIS(30) = 0.30',
 'MIS(40) = 0.40',
 'MIS(50) = 0.40',
 'MIS(60) = 0.30',
 'MIS(70) = 0.20',
 'MIS(80) = 0.20',
 'MIS(90) = 0.20',
 'MIS(100) = 0.10',
 'MIS(120) = 0.20',
 'MIS(140) = 0.15',
 'SDC = 0.1',
 'cannot_be_together: {20, 40}, {70, 80}',
 'must-have: 20 or 40 or 50']

In [3]:
ms = pd.Series(dict([(int(j[0][4:-1]), float(j[1])) for j in [i.split(" = ") for i in argl if i.startswith("MIS")]])).sort_values().reset_index()
ms

,index,0
0,100,0.10
1,140,0.15
2,70,0.20
3,80,0.20
4,90,0.20
5,120,0.20
6,20,0.30
7,30,0.30
8,60,0.30
9,40,0.40


In [4]:
id_dict = ms["index"].to_dict()
id_dict_inv = {val: key for key, val in id_dict.items()}
id_dict, id_dict_inv

({0: 100,
  1: 140,
  2: 70,
  3: 80,
  4: 90,
  5: 120,
  6: 20,
  7: 30,
  8: 60,
  9: 40,
  10: 50,
  11: 10},
 {10: 11,
  20: 6,
  30: 7,
  40: 9,
  50: 10,
  60: 8,
  70: 2,
  80: 3,
  90: 4,
  100: 0,
  120: 5,
  140: 1})

In [5]:
ms_dict = ms[0].to_dict()
ms_dict

{0: 0.10000000000000001,
 1: 0.14999999999999999,
 2: 0.20000000000000001,
 3: 0.20000000000000001,
 4: 0.20000000000000001,
 5: 0.20000000000000001,
 6: 0.29999999999999999,
 7: 0.29999999999999999,
 8: 0.29999999999999999,
 9: 0.40000000000000002,
 10: 0.40000000000000002,
 11: 0.42999999999999999}

In [6]:
s = pd.read_csv("proj1_input-data.txt", header = None, sep = "\t",squeeze = True)
s

0                   {20, 30, 80, 70, 50, 90}
1                           {20, 10, 80, 70}
2                               {10, 20, 80}
3                               {20, 30, 80}
4                                   {20, 80}
5    {20, 30, 80, 70, 50, 90, 100, 120, 140}
Name: 0, dtype: object

In [7]:
da = s.str[1:-1].str.get_dummies(sep = ", ").rename(columns = lambda x: id_dict_inv[int(x)]).reindex(columns = ms.index, fill_value = 0).values
da

array([[0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0]])

In [65]:
def mis(L):
    op = [min([ms_dict[j] for j in i]) for i in L]
    return(op)

In [66]:
def sup(L):
    op = np.vstack(da[:, i].all(axis = 1) for i in L).mean(axis = 1)
    return(op)

In [67]:
Li = (da.mean(axis = 0) > ms[0].values).argmax()
L = np.array([ms.loc[Li:, 0].index[da[:, Li:].mean(axis = 0) > ms[0][Li]]]).T
Li, L

(0, array([[ 0],
        [ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5],
        [ 6],
        [ 7],
        [10],
        [11]]))

In [68]:
Lsup = sup(L)
Lsup

array([ 0.16666667,  0.16666667,  0.5       ,  1.        ,  0.33333333,
        0.16666667,  1.        ,  0.5       ,  0.33333333,  0.33333333])

In [69]:
Lmis = mis(L)
Lmis

[0.10000000000000001,
 0.14999999999999999,
 0.20000000000000001,
 0.20000000000000001,
 0.20000000000000001,
 0.20000000000000001,
 0.29999999999999999,
 0.29999999999999999,
 0.40000000000000002,
 0.42999999999999999]

In [70]:
F = [L[Lsup > Lmis]]
F

[array([[0],
        [1],
        [2],
        [3],
        [4],
        [6],
        [7]])]

In [71]:
pd.DataFrame({"MIS": Lmis, "Support": Lsup}, index = ms.loc[L.T[0], "index"])

,MIS,Support
index,,
100,0.10,0.166667
140,0.15,0.166667
70,0.20,0.500000
80,0.20,1.000000
90,0.20,0.333333
120,0.20,0.166667
20,0.30,1.000000
30,0.30,0.500000
50,0.40,0.333333


In [86]:
phi = 1
def pair_sup_mis(L):
    Lsup = sup(L)
    Lmis = mis(L)
    Lsup_t = Lsup[:, np.newaxis]
    C = sp.coo_matrix(np.triu((Lsup_t >= Lmis).T & (np.abs(Lsup_t - Lsup) < phi), 1)).nonzero()
    op = np.array([L.T[0][i] for i in C]).T
    return(op)
def l2_candidate_gen(L):
    C = pair_sup_mis(L)
    C = C[np.in1d(C[:,0], F[0])]
    return(C)

In [ ]:
def reduce_candicate(C):
    

In [87]:
C = l2_candidate_gen(L)
C

array([[ 0,  1],
       [ 0,  2],
       [ 0,  3],
       [ 0,  4],
       [ 0,  5],
       [ 0,  6],
       [ 0,  7],
       [ 0, 10],
       [ 0, 11],
       [ 1,  2],
       [ 1,  3],
       [ 1,  4],
       [ 1,  5],
       [ 1,  6],
       [ 1,  7],
       [ 1, 10],
       [ 1, 11],
       [ 2,  3],
       [ 2,  4],
       [ 2,  6],
       [ 2,  7],
       [ 2, 10],
       [ 2, 11],
       [ 3,  4],
       [ 3,  6],
       [ 3,  7],
       [ 3, 10],
       [ 3, 11],
       [ 4,  6],
       [ 4,  7],
       [ 4, 10],
       [ 4, 11],
       [ 6,  7],
       [ 6, 10],
       [ 6, 11],
       [ 7, 10],
       [ 7, 11]])

In [74]:
F.append(C[sup(C) >= mis(C)])
F[-1]

array([[ 0,  1],
       [ 0,  2],
       [ 0,  3],
       [ 0,  4],
       [ 0,  5],
       [ 0,  6],
       [ 0,  7],
       [ 0, 10],
       [ 1,  2],
       [ 1,  3],
       [ 1,  4],
       [ 1,  5],
       [ 1,  6],
       [ 1,  7],
       [ 1, 10],
       [ 2,  3],
       [ 2,  4],
       [ 2,  6],
       [ 2,  7],
       [ 2, 10],
       [ 3,  4],
       [ 3,  6],
       [ 3,  7],
       [ 3, 10],
       [ 3, 11],
       [ 4,  6],
       [ 4,  7],
       [ 4, 10],
       [ 6,  7],
       [ 6, 10],
       [ 6, 11],
       [ 7, 10]])

In [180]:
Ls = pd.DataFrame(F[-1][:, -1], pd.MultiIndex.from_tuples(F[-1][:, :-1].tolist()))

In [181]:
grp = Ls.groupby(level = [0])

In [183]:
Ls

,0
0,1
0,2
0,3
0,4
0,5
0,6
0,7
0,10
1,2
1,3


In [182]:
grp.apply(pair_sup_mis)

ValueError: 'axis' entry is out of bounds

In [97]:
np.tile(Lbase, (10, 1))

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

In [47]:
np.array([Ldiff[i] for i in sup_mis(sup(Ldiff), mis(Ldiff))]).T

array([[[ 1,  2],
        [ 1,  3],
        [ 1,  4],
        [ 1,  5],
        [ 1,  6],
        [ 1,  7],
        [ 1, 10],
        [ 2,  3],
        [ 2,  4],
        [ 2,  6],
        [ 2,  7],
        [ 2, 10],
        [ 3,  4],
        [ 3,  6],
        [ 3,  7],
        [ 3, 10],
        [ 4,  6],
        [ 4,  7],
        [ 4, 10],
        [ 5,  6],
        [ 5,  7],
        [ 5, 10],
        [ 6,  7],
        [ 6, 10],
        [ 7, 10]]])

In [14]:
def ms_candidate_gen(F, phi = 1):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-14-d6d9bdbf8167>, line 2)